In [ ]:
import Analysis_Code.discrete_quantum as dq
from Analysis_Code.parameter_sweep import *
from Analysis_Code.useful import *
from Analysis_Code.iplots import *

from numpy import empty, arange, zeros, mean, std, linspace, random, diag, array, ones, logspace
import numpy as np
from numba import njit
from timeit import *
from tqdm import tqdm

from unipolator import *
from kronbinations import *

import os
import matplotlib.pyplot as plt
#import tikzplotlib
plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    "pgf.preamble": "\n".join([
         r"\usepackage[utf8x]{inputenc}",
         r"\usepackage[T1]{fontenc}"])})

In [ ]:
# 1D interpolation but with states
n = 1 # 1D interpolation
s = 16
n_times = 128

bins = array([1,2,4,8])
mini = 0.0
maxi = 1.0
rng = random.default_rng(123)
markersize = 4
plot_log = 1 # 1=semilogy; 2=loglog
ps = 0.68
both_plots = False

c_mins = ones(n)*mini
c_maxs = ones(n)*maxi

how_many = 8*15+1
c_s = linspace(mini,maxi,how_many)
lim2 = 1.0
n_bins = len(bins)

I_ui, I_sym_ui, I_trotter, I_sym_trotter, I_classic = empty([ n_bins, how_many]), empty([ n_bins, how_many]), empty([how_many]), empty([how_many]), empty([how_many])

amps = np.pi/2 * array([1.0, 0.025])
#H_s = dq.Random_parametric_Hamiltonian_Haar(n, s, amps, rng=rng)
H_s = dq.Random_parametric_Hamiltonian_gauss(n, s, sigmas=amps, rng=rng)
v0 = dq.rand_state(s, rng=rng)

U_ui = empty([s,s], dtype=complex)
U_classic = empty([s,s], dtype=complex)
v_classic = empty([s,1], dtype=complex)
v_classic2 = empty([s,1], dtype=complex)
v_out_ui = empty([s,1], dtype=complex)
v_out_sym_ui = empty([s,1], dtype=complex)
v_out_trotter = empty([s,1], dtype=complex)
v_out_sym_trotter = empty([s,1], dtype=complex)
ui, sym_ui  = [], []
#for i, bin in enumerate(bins):
#    c_bins = bin*np.ones(n, dtype=int)
#    # Prepare interpolations and exact decompositions
#    ui.append(UI_vector(H_s, c_mins, c_maxs, c_bins))
#    sym_ui.append(Sym_UI_vector(H_s, c_mins, c_maxs, c_bins))
system = Hamiltonian_System(H_s)
system_vector = Hamiltonian_System_vector(H_s)
system_vector.set_m_star(20)
trotter = Trotter_System_vector(H_s, n_times=n_times)
sym_trotter = Sym_Trotter_System_vector(H_s, n_times=n_times)

for i, d in enumerate(c_s):
    c = np.array([d]) #np.random.random(n)
    system.expmH(c, U_classic)
    v_classic = U_classic @ v0
    system_vector.expmH(c, v0, v_classic2)
    I_classic[i] = np.abs(dq.State_Infidelities(v_classic, v_classic2))   # resulting vectors are not normalized -> infidelity can get negative
    trotter.expmH(c, v0, v_out_trotter)
    I_trotter[i] = dq.State_Infidelities(v_out_trotter, v_classic)
    sym_trotter.expmH(c, v0, v_out_sym_trotter)
    I_sym_trotter[i] = dq.State_Infidelities(v_out_sym_trotter, v_classic)
#    for j in range(len(bins)):
#        ui[j].expmH(c, v0, v_out_ui)#, dv_out)
#        sym_ui[j].expmH(c, v0, v_out_sym_ui)
#        I_ui[j,i] = dq.State_Infidelities(v_out_ui, v_classic)
#        I_sym_ui[j,i] = dq.State_Infidelities(v_out_sym_ui, v_classic)
#
fig, axs = plt.subplots(1,1, figsize=(10,10))
plt.semilogy(c_s, I_trotter, label='Trotter')
plt.semilogy(c_s, I_sym_trotter, label='Sym Trotter')
plt.semilogy(c_s, I_classic, label='Exact')
#for i in range(len(bins)):
#    plt.semilogy(c_s, I_ui[i,:], label='UI ('+str(bins[i])+' bins)')
#for i in range(len(bins)):
#    plt.semilogy(c_s, I_ui[i,:], label='Sym UI ('+str(bins[i])+' bins)', linestyle='dashed')
## add legend
plt.legend(loc='upper right', fontsize=20)

In [ ]:
# 2D interpolation but with states
n = 2 # 1D interpolation
s = 16
bin = 3
mini = 0.0
maxi = 1.0
do_figures = 1
rng = random.default_rng(123)
markersize = 4
width = 8.5
ps = 0.68
n_times = 32


U_classic = empty([s,s], dtype=complex)
v_classic = empty([s,1], dtype=complex)
v_trotter = empty([s,1], dtype=complex)
v_sym_trotter = empty([s,1], dtype=complex)
how_many = 20
c_s = linspace(0,1,how_many)

I_trotter = empty([how_many, how_many])
I_sym_trotter = empty([how_many, how_many])

amps = np.pi/2*array([1.0, 0.025])
H_s = dq.Random_parametric_Hamiltonian_gauss(n, s, sigmas=amps, rng=rng)
system = Hamiltonian_System(H_s)
trotter = Trotter_System_vector(H_s, n_times=n_times)
sym_trotter = Sym_Trotter_System_vector(H_s, n_times=n_times)
v0 = dq.rand_state(s, rng=rng)
c_bins = bin*np.ones(n, dtype=int)
for i, d in enumerate(c_s):
    for j, d2 in enumerate(c_s):
        c = np.array([d, d2]) #np.random.random(n)
        system.expmH(c, U_classic)
        v_classic = U_classic @ v0
        trotter.expmH(c, v0, v_trotter)
        I_trotter[i,j] = dq.State_Infidelities(v_trotter, v_classic)
        sym_trotter.expmH(c, v0, v_sym_trotter)
        I_sym_trotter[i,j] = dq.State_Infidelities(v_sym_trotter, v_classic)

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,10))
im = ax1.imshow(I_trotter[:,:], extent=[mini,maxi,mini,maxi], origin='lower', aspect='auto')
cbar = fig.colorbar(im, ax=ax1)
im2 = ax2.imshow(I_sym_trotter[:,:], extent=[mini,maxi,mini,maxi], origin='lower', aspect='auto')
cbar2 = fig.colorbar(im2, ax=ax2)

In [ ]:
# 2D interpolation but with states
n = 2 # 1D interpolation
s = 16
bin = 1
mini = 0.0
maxi = 1.0
do_figures = 1
rng = random.default_rng(123)
markersize = 4
width = 8.5
ps = 0.68

c_mins = zeros(n)
c_maxs = ones(n)
c_bins = bin*np.ones(n, dtype=int)

U_classic = empty([s,s], dtype=complex)
v_classic = empty([s,1], dtype=complex)
v_ui = empty([s,1], dtype=complex)
v_sym_ui = empty([s,1], dtype=complex)
how_many = 20
c_s = linspace(0,1,how_many)

I_ui = empty([how_many, how_many])
I_sym_ui = empty([how_many, how_many])

amps = np.pi/2*array([1.0, 0.025])
H_s = dq.Random_parametric_Hamiltonian_gauss(n, s, sigmas=amps, rng=rng)
system = Hamiltonian_System(H_s)
ui = UI_vector(H_s, c_mins, c_maxs, c_bins)
sym_ui = Sym_UI_vector(H_s, c_mins, c_maxs, c_bins)
v0 = dq.rand_state(s, rng=rng)
c_bins = bin*np.ones(n, dtype=int)
for i, d in enumerate(c_s):
    for j, d2 in enumerate(c_s):
        c = np.array([d, d2]) #np.random.random(n)
        system.expmH(c, U_classic)
        v_classic = U_classic @ v0
        ui.expmH(c, v0, v_ui)
        I_ui[i,j] = dq.State_Infidelities(v_ui, v_classic)
        sym_ui.expmH(c, v0, v_sym_ui)
        I_sym_ui[i,j] = dq.State_Infidelities(v_sym_ui, v_classic)

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,10))
im = ax1.imshow(I_ui[:,:], extent=[mini,maxi,mini,maxi], origin='lower', aspect='auto')
cbar = fig.colorbar(im, ax=ax1)
im2 = ax2.imshow(I_sym_ui[:,:], extent=[mini,maxi,mini,maxi], origin='lower', aspect='auto')
cbar2 = fig.colorbar(im2, ax=ax2)


In [ ]:
checksum_pre = 'pgi_'
## Calculate the time it takes to get to a target fidelity via kronbinations parameter sweep
redo = False
# Variation by different variables, 
# specify the variables by a dictionary 
num_controls = {'label': 'Number of controls', 'name': 'num_controls', 'def_val': 2, 'variation_vals': np.arange(1, 3), 'variation_type': 'int'}
hilbert_dim = {'label':'Hilbert space dimension', 'name': 'dim_hilbert', 'def_val': 16, 'variation_vals': 2**np.arange(1, 10), 'variation_type': 'log_int'}
control_amplitudes = {'label': 'Eigenvalue ratio', 'name': 'amp_ratio', 'def_val': 0.025, 'variation_vals': np.logspace(-3, 0, 61), 'variation_type': 'log'} # ([H_i>0| / |H_0|)
bins = {'label': 'Number of bins', 'name': 'bins', 'def_val': 1, 'variation_vals': np.arange(1,64), 'variation_type': 'log_int'}
all_parameter_dicts = [num_controls, hilbert_dim, control_amplitudes, bins] 
# Methods to use 
m_times_list_ui = [1,2,4,8,16,32,64,128] 
m_times_list_multiply    = [i for i in range(25)]
m_times_list_trotter = list(np.round(np.logspace(0,np.log10(1024),16)).astype(int))
m_times_list_sym_trotter = list(np.round(np.logspace(0,np.log10(128),11)).astype(int))
method_dict = {}
method_dict['Expm Multiply'] = {'method': Hamiltonian_System_vector, 'm_times': m_times_list_multiply}
method_dict['UI'] = {'method': UI_vector, 'm_times': m_times_list_ui}
method_dict['Sym UI'] = {'method': Sym_UI_vector, 'm_times': m_times_list_ui}
method_dict['Trotter'] = {'method': Trotter_System_vector, 'm_times': m_times_list_trotter}
method_dict['Sym Trotter'] = {'method': Sym_Trotter_System_vector, 'm_times': m_times_list_sym_trotter}

import_statements = ['Analysis_Code.Average_Infidelity_Integrator', 'kronbinations', 'unipolator', ['Analysis_Code.discrete_quantum', 'dq'], 'from numba import jit', 'import timeit']
other_func = [timeit_autorange_vector, initialize_rep_arrays, intitialize_integrators]
combinations = ['num_controls']  
to_do = ['mean', 'times'] 
for i, combination in enumerate(combinations):
    parameters, axis_labels, axis_types = get_parameters(combination, all_parameter_dicts)  
    k = JIT_kronbinations(parameters, func=Binned_Infidelities_and_Times_vector, other_func=other_func, import_statements=import_statements, other_arguments=[method_dict, to_do], redo=redo, checksum_pre=checksum_pre)
    method_dict_results, method_dict = create_variables(k, method_dict, to_do=to_do)
    print(k.checksum)
    k.calculate_all()
    line('Done with '+combination)

In [ ]:
import scipy
# plot the infidelity as a function of the number of trotter steps and the number of bins to reach a target fidelity  --> load the apropatirate data ('amp_ratio')
# at which amp_ratio to plot ? --> allow change via slider
f= lambda x, a, b: a + b*np.log(x)
df = lambda x, a, b, da, db: np.sqrt(da**2 + (np.log(x)*db)**2)
f_trotter = lambda x, a, b, c: a + b*x**c
df_trotter = lambda x, a, b, c, da, db, dc: np.sqrt(da**2 + (x**c*db)**2 + (b*c*x**(c-1))**2)
letters = ['a', 'b']
min_x = 10**-16 
for num_controls_index, num_controls in enumerate([1, 2]):
    # for every method, plot the infidelity as a function of the number of trotter steps (or the number of bins if UI)
    ps = 0.68
    legendary = []
    min_x = 10**-15
    lineparameters = {} # fit a line to the data (in logarithm, so t(I) = a + b*exp(I)

    for plot_which in ['mean']: #, 'max']:
        line(checksum_pre[:-1] + ' ' + plot_which)
        fig, ax = plt.subplots(1,1, figsize=(8,8))
        # Determine the maximum value of the infidelity
        indexes = []
        for d, (name, data) in enumerate(method_dict_results.items()):
            if not name == 'times':
                # just generate the legend 
                ax.plot([min_x/10, min_x/5], [10**-6, 10**-6], label=name, color='C'+str(d))
        for d, (name, data) in enumerate(method_dict_results.items()):
            if not name == 'times':
                # extract values
                t_vals = data['times'][num_controls_index,:]
                # Calculate the partial minima 
                mean_I, std_I_min, std_I_max = data[plot_which+'_I_mean'][num_controls_index,:], data[plot_which+'_I_std_min'][num_controls_index,:], data[plot_which+'_I_std_max'][num_controls_index,:]
                # Calculate the partial minima 
                # Double the va
                mean_I = np.abs(mean_I)
                std_I_min = np.abs(std_I_min)
                std_I_max = np.abs(std_I_max)
                mean_indexes = index_partial_minimum(mean_I)#, std_I_min)
                indexes.append(mean_indexes)
                mean_Is = mean_I[mean_indexes]
                min_Is =  std_I_min[mean_indexes] 
                max_Is =  std_I_max[mean_indexes]
                ts = t_vals[mean_indexes]
                # without line connecting draw errorbars
                for i in range(len(mean_Is)):
                    if mean_Is[i] - min_Is[i] < min_x:
                        if mean_Is[i] < min_x:
                            min_Is[i] = mean_Is[i] - min_x
                        else:
                            min_Is[i] = 0.0
                min_Is = abs(min_Is)
                ax.errorbar(mean_Is, ts, xerr=(min_Is, max_Is), capsize=5, capthick=2, fmt='o', markersize=2, markeredgewidth=2, color='C'+str(d)) #label=name, 
                # fit f to the data using scipy.optimize.curve_fit
                curr_f = f
                curr_p0 = [0.1,0.1]
                if 'Trotter' in name:
                    curr_f = f_trotter
                    curr_p0 = [0.0,0.001,-0.3]
                if 'Expm' in name:
                    # find first index with mean_Is < 10**-5
                    ind = np.where(mean_Is < 10**-5)[0][0]
                elif 'Trotter' in name:
                    ind = 1
                else:
                    ind = 0
                fit_param, covariance = scipy.optimize.curve_fit(curr_f, mean_Is[ind:], ts[ind:], p0=curr_p0)
                std_param = np.sqrt(np.diag(covariance))
                # save in lineparameters
                lineparameters[name] = {'param': fit_param, 'std_param': std_param}
        for d, (name, data) in enumerate(method_dict_results.items()):
            if not name == 'times':
                curr_f = f
                curr_df = df
                if 'Trotter' in name:
                    curr_f = f_trotter
                    curr_df = df_trotter
                I = np.abs(data[plot_which+'_I_mean'][num_controls_index,indexes[d]])
                min_I, max_I = np.min(I), np.max(I)
                if 'Expm' in name:
                    # find first index with mean_Is < 10**-5
                    ind = np.where(I < 10**-5)[0][0]
                    max_I = np.abs(I[ind])
                elif 'Trotter' in name:
                    max_I = np.abs(I[1])
                
                param = lineparameters[name]['param']
                dparam = lineparameters[name]['std_param']
                
                #x_min = np.array([min_x, min_I]) 
                x = np.logspace(np.log10(min_x), np.log10(max_I), 30)
                # with color c+d
                curr_ys = curr_f(x, *param)
                # if value too large >1.0, remove it (also remove x)
                curr_ys = curr_ys[curr_ys < 1.0]
                x = x[:len(curr_ys)]
                ax.semilogx(x, curr_ys, linestyle=':', color='C'+str(d))
                # evaluate error at 10**-15 (min_x) using error propagation
                t_at_min_I = curr_f(min_I, *param)
                #dt_at_min_I = curr_df(min_I, *param, *dparam)
                #print(name+': a = ' + plusminus(a, da,1) + ', b = ' + plusminus(b, db,1) + ', t(I=10**-15) = ' + plusminus(t_at_min_I, dt_at_min_I,1)) 
        ax.set_ylabel(r't [s]')
        ax.set_xscale('log')
        #ax.set_yscale('log')
        ax.set_xlabel(r'$I=1-F$')
        ax.legend(*legendary, loc='best')
        xlim = ax.get_xlim()
        ax.set_xlim([min_x, xlim[1]])
        ylim = ax.get_ylim()
        ax.set_ylim([0, 2.5*10**-5])
        ax.set_xlim([10**-15,1.0])
        # show figure
        plt.show()
        filename = checksum_pre+plot_which+'_time2infidelity_vector_'+str(num_controls)
        #fig2tikz(fig, filename, add_letter=letters[num_controls_index])


In [ ]:
#file_list = ['pgi_mean_time2infidelity_vector_1',
#'pgi_mean_time2infidelity_vector_2']

#output_filename = 'time2infidelity_vectors'
#combine_tex_files(file_list, output_filename, columns=2, directory='Tikz', redo=True, shift_by_y=0.5, columns_half=1.18)